In [1]:
import numpy as np
import importlib
import util as uti
import matplotlib.pyplot as plt

# import math lib
from math import pi

# import Qiskit
from qiskit import Aer, IBMQ, execute, assemble, transpile
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.quantum_info import state_fidelity, Statevector

# import basic plot tools
#import qiskit.visualization as vis
from qiskit.visualization import plot_bloch_multivector, plot_histogram
from qiskit_textbook.tools import vector2latex
%matplotlib inline

/home/kinga/.local/lib/python3.6/site-packages/qiskit/__init__.py:66: DeprecationWarning: Using Qiskit with Python 3.6 is deprecated as of qiskit-terra 0.17.0. Support for running Qiskit with Python 3.6 will be removed in qiskit-terra 0.20.0.
  DeprecationWarning,


In [2]:
def calc_rotations(vv):
    return (vv + 1) * pi / 2

In [ ]:
# basic swap test directly on quantum states that are rotated against each other
def swap_test(rot_a, rot_b):
    
    backend = Aer.get_backend('qasm_simulator')
    # create circuit with n qubits and one classical bit for measurement
    # qubits: first: ancilla, second: input vector, third: cluster center
    #(for latent space in R^2: 1 (ancilla) + 1 (x1,x2 coords of input) + 1 (x1,x2 coords of cluster))
    n = 2+1
    qc = QuantumCircuit(n, 1, name="swap_test")

    # control qubit default 0
    # rotate qubit a around x by rotate_a theta
    qc.u(rot_a[0], rot_a[1], 0, 1)
    # rotate qubit b around x by rotate_b theta
    qc.u(rot_b[0], rot_b[1], 0, 1, 0, 2)
    # third qubit init to phi
    qc.barrier()
    # apply hadamard to control
    qc.h(0)
    # swap psi ancilla with phi
    qc.cswap(0,1,2)
    # apply second hadamard
    qc.h(0)
    # measure control qubit
    qc.measure(0,0)
    qc.draw()
    
    shots = 1024
    qobj = assemble(qc, backend)
    results = backend.run(qobj).result()
    answer = results.get_counts()

    return qc, answer